In [ ]:
from keras.models import Sequential # basic class for specifying and training a neural network
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Activation
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import nadam
import cv2
import numpy as np
from os import listdir
import tensorflow as tf
from matplotlib import pyplot as plt

In [ ]:
batch_size = 5
num_epochs = 100
kernel_size = 3 # using 3x3 kernels throughout
pool_size = 2 # using 2x2 pooling throughout
conv_depth_1 = 16 # kernels per conv. layer 1
conv_depth_2 = 32 # kernels per conv. layer 1 after the first pooling layer
drop_prob_1 = 0.5 # dropout after pooling5
drop_prob_2 = 0.5 # dropout in the FC layer
hidden_size = 128 # neurons for FC layer
max_items_for_class = 5000 # count of items of each class in dataset
class_one_dir = 'Data/womens'
class_two_dir = 'Data/mans'
img_size = (64, 64)

In [ ]:
def shuffle_in_unison(a, b):
    # courtsey http://stackoverflow.com/users/190280/josh-bleecher-snyder
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [ ]:
def create_Xt_Yt(X, y, percentage=0.9):
    p = int(len(X) * percentage)
    X_train = X[0:p]
    Y_train = y[0:p]

    X_test = X[p:]
    Y_test = y[p:]

    return X_train, X_test, Y_train, Y_test

In [ ]:
def make_x_y():
    x, y = [], []
    for i, img in enumerate(listdir(class_one_dir)):  # iteration over images in a folder
        try:
                if i >= max_items_for_class:
                    break
                img = cv2.imread(class_one_dir+'/'+img, cv2.IMREAD_ANYCOLOR)
                res = cv2.resize(img, img_size)
                x.append(res)
                y.append(1)
        except Exception as e:
            print(e)
            print(i)

    for i, img in enumerate(listdir(class_two_dir)):  # iteration over images in a folder
        try:
                if i >= max_items_for_class:
                    break
                img = cv2.imread(class_two_dir+'/'+img, cv2.IMREAD_ANYCOLOR)
                res = cv2.resize(img, img_size)
                x.append(res)
                y.append(0)
        except Exception as e:
            print(e)
            print(i)
    x = np.array(x)
    y = np.array(y)
    x, y = shuffle_in_unison(x, y) #  shuffle dataset
    x_train, x_test, y_train, y_test = create_Xt_Yt(x, y)
    return (np.array(x_train), np.array(y_train)), (np.array(x_test), np.array(y_test))

In [ ]:
(x_train, y_train), (x_test, y_test) = make_x_y()
(num_train), (depth, height, width) = (x_train.shape[0]), (x_train[0].shape)
num_test = x_test.shape[0]
num_classes = np.unique(y_train).shape[0]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= np.max(x_train) # Normalise data to [0, 1] range
x_test /= np.max(x_train) # Normalise data to [0, 1] range

y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(conv_depth_1, (kernel_size, kernel_size), padding='same', input_shape=(depth, height, width)))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Conv2D(conv_depth_1, (kernel_size, kernel_size), padding='same'))
model.add(LeakyReLU())

model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
model.add(Dropout(drop_prob_1))    

model.add(Conv2D(conv_depth_2, (kernel_size, kernel_size), padding='same'))
model.add(LeakyReLU())
model.add(Conv2D(conv_depth_2, (kernel_size, kernel_size), padding='same'))
model.add(LeakyReLU())
          
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
          
model.add(Flatten())
          
model.add(Dense(hidden_size))
model.add(LeakyReLU())
model.add(Dropout(drop_prob_2))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
opt = nadam(lr=0.0001) # usung nadam optimizer
checkpointer = ModelCheckpoint(filepath="weights.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train,                # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1,  callbacks=[checkpointer]) # ...holding out 10% of the data for validation

Visualisation of learning process

In [ ]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

In [ ]:
model.load_weights("weights.h5")
pred = model.predict(x_test)
pp = tf.placeholder('float', [None, 2])
yy = tf.placeholder('float', [None, 2])
correct = tf.equal(tf.argmax(pp, 1), tf.argmax(yy, 1))
accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
with tf.Session() as sess:
    print('Accuracy:', accuracy.eval({pp: pred, yy: y_test}))